In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import dask
import os, sys
import glob
import zarr

sys.path.append('/')
from libraries import *

- My initial idea has been to combine wind speed data at analysis and forecast times.
- However, I mistakenly downloaded analysis data for remaining vertical levels, rather than forecast. 
- I can still download the forecast data, but it takes too much time. 
- Thus, I first combine the analysis data, across height levels. 

In [2]:
print("Starting parallel computing...")
import dask.distributed as dd
cluster = dd.LocalCluster(n_workers=24,threads_per_worker=2,memory_limit='4GB',dashboard_address='8787')
# Connect to the cluster
client = dd.Client(cluster)
print(client)

Starting parallel computing...
<Client: 'tcp://127.0.0.1:46207' processes=24 threads=48, memory=89.41 GiB>


In [3]:
def preprocess(ds):
    ds['time'] = ds['valid_time']
    ds = ds.drop(['valid_time', 'step', 'latitude','longitude'])
    return ds
def preprocess_2(ds):
    '''
    This script process the remaining height level forecast data
    '''
    ds = ds.rename({'valid_time':'time'})
    ds = ds.drop(['expver','latitude','longitude'])
    return ds

# Initializing a zarr by reading sample data
- Once created, no need to repeat again.

dates = pd.date_range(start='2011-01-01T00', end='2020-12-31T23', freq='h')
zarr_store = '/data/harish/CERRA_wind_profiles_and_Chebyshev_coefficients/CERRA_height_level_winds.zarr'
def template_zarr_init(zarr_store, dates):
    ds = xr.open_dataset(f'/media/harish/External_3/CERRA_ws_15_30_50_75_200_250_300_400_500/2020/CERRA_gridded_15_30_50_75_200_250_300_400_500_wind_2020_1.nc').ws
    ds = preprocess(ds)
    template = ds.chunk({'time': 1,'y': -1,'x': -1, 'heightAboveGround':1}).pipe(xr.zeros_like).isel(time=0,heightAboveGround=0,
                                                                        drop=True).expand_dims(time=len(dates), heightAboveGround=len(CERRA_levels))
    template['time'] = dates
    template['heightAboveGround'] = CERRA_levels
    template = template.chunk({'time': 1,'heightAboveGround':1})
    template.to_dataset(name = 'wind_speed').to_zarr(zarr_store, compute=False, consolidated=True, mode='w')
    return template
'''
Initialize the zarr store, which creates the zarr store in disk, with zeros. 
Once created, better to chose append mode for further operations or else it will overwrite the existing data.
'''
template_zarr_init(zarr_store, dates)

In [6]:
year = 2020
month = 1
def read_monthly_data(year,month):
    chunks = {'time': 8}
    ds_10m = xr.open_dataset(f'/media/harish/External_3/CERRA_ws10/{year}/CERRA_{year}_{month}.nc',chunks = chunks).si10
    ds_10m = preprocess(ds_10m)
    ds_10m_1 = xr.open_dataset(f'/media/harish/External_3/CERRA_ws10_step1/{year}/CERRA_{year}_{month}.nc',chunks = chunks).si10
    ds_10m_1 = preprocess(ds_10m_1)
    ds_10m_2 = xr.open_dataset(f'/media/harish/External_3/CERRA_ws10_step2/{year}/CERRA_{year}_{month}.nc',chunks = chunks).si10
    ds_10m_2 = preprocess(ds_10m_2)

    ds_100m = xr.open_dataset(f'/media/harish/External_3/CERRA_ws100/{year}/CERRA_gridded_100_m_wind_{year}_{month}.nc',chunks = chunks).ws
    ds_100m = preprocess(ds_100m)
    ds_150m = xr.open_dataset(f'/media/harish/External_3/CERRA_ws150/{year}/CERRA_gridded_150_m_wind_{year}_{month}.nc',chunks = chunks).ws
    ds_150m = preprocess(ds_150m)
    ds_100_150m_1 = xr.open_dataset(f'/media/harish/External_3/CERRA_ws_100_150_step1/{year}/CERRA_gridded_wind_{year}_{month}_1.nc',chunks = chunks).ws
    ds_100_150m_1 = preprocess(ds_100_150m_1)
    ds_100_150m_2 = xr.open_dataset(f'/media/harish/External_3/CERRA_ws_100_150_step2/{year}/CERRA_gridded_wind_{year}_{month}_2.nc',chunks = chunks).ws
    ds_100_150m_2 = preprocess(ds_100_150m_2)

    ds_remaining_height = xr.open_dataset(f'/media/harish/External_3/CERRA_ws_15_30_50_75_200_250_300_400_500/{year}/CERRA_gridded_15_30_50_75_200_250_300_400_500_wind_{year}_{month}.nc',chunks = chunks).ws
    ds_remaining_height = preprocess(ds_remaining_height)
    chunks = {'valid_time':8}
    ds_remaining_height_1 = xr.open_dataset(f'/media/harish/External_3/CERRA_ws_15_30_50_75_200_250_300_400_500_step1/{year}/CERRA_gridded_15_30_50_75_200_250_300_400_500_wind_{year}_{month}_1.nc',chunks = chunks).ws
    ds_remaining_height_1 = preprocess_2(ds_remaining_height_1)
    ds_remaining_height_2 = xr.open_dataset(f'/media/harish/External_3/CERRA_ws_15_30_50_75_200_250_300_400_500_step2/{year}/CERRA_gridded_15_30_50_75_200_250_300_400_500_wind_{year}_{month}_2.nc',chunks = chunks).ws
    ds_remaining_height_2 = preprocess_2(ds_remaining_height_2)
    return [ds_10m,ds_10m_1,ds_10m_2,ds_100m,ds_150m,ds_100_150m_1,ds_100_150m_2,ds_remaining_height,ds_remaining_height_1,ds_remaining_height_2]


In [57]:
dates = pd.date_range(start='2011-01-01T00', end='2020-12-31T23', freq='h')
zarr_store = '/data/harish/CERRA_wind_profiles_and_Chebyshev_coefficients/CERRA_height_level_winds.zarr'
def write_to_zarr(ds, zarr_store):
    time_indices = np.searchsorted(dates.values, ds.time.values)
    height_indices = np.atleast_1d(np.searchsorted(CERRA_levels, ds.heightAboveGround.values))
    print(ds.time.values[0],ds.time.values[-1],ds.heightAboveGround.values)
    print(time_indices[0],time_indices[-1],height_indices)

    for i, time_idx in enumerate(time_indices):
        if len(height_indices) > 1:
            for j,height_idx in enumerate(height_indices):
                # Define the specific region for one time step and one height level
                region = {
                    "time": slice(time_idx, time_idx + 1),  # Single time index
                    "heightAboveGround": slice(height_idx, height_idx + 1),  # Single height index
                }

                # Select one time step and one height level from ds_remaining_height
                ds_chunk = (
                    ds.isel(time=i, heightAboveGround=j)
                    .expand_dims(dim={"time": [ds.time.values[i]], 
                                    "heightAboveGround": [ds.heightAboveGround.values[j]]})
                    .to_dataset(name="wind_speed")
                )

                # Write to Zarr using the specified region
                ds_chunk.to_zarr(zarr_store, region=region, mode="r+")
        else:
            region = {
                    "time": slice(time_idx, time_idx + 1),  # Single time index
                    "heightAboveGround": slice(height_indices[0], height_indices[0] + 1),  # Single height index
                }
            ds_chunk = (
                    ds.isel(time=i)
                    .expand_dims(dim={"time": [ds.time.values[i]],
                                        "heightAboveGround": [ds.heightAboveGround.values]})
                    .to_dataset(name="wind_speed")
                )
            ds_chunk.to_zarr(zarr_store, region=region, mode="r+")


In [58]:
ds_stores = read_monthly_data(year,month)
for ds in ds_stores:
    write_to_zarr(ds, zarr_store)

2020-01-01T00:00:00.000000000 2020-01-31T21:00:00.000000000 10.0
78888 79629 [0]
2020-01-01T01:00:00.000000000 2020-01-31T22:00:00.000000000 10.0
78889 79630 [0]
2020-01-01T02:00:00.000000000 2020-01-31T23:00:00.000000000 10.0
78890 79631 [0]
2020-01-01T00:00:00.000000000 2020-01-31T21:00:00.000000000 100.0
78888 79629 [5]
2020-01-01T00:00:00.000000000 2020-01-31T21:00:00.000000000 150.0
78888 79629 [6]
2020-01-01T01:00:00.000000000 2020-01-31T22:00:00.000000000 [100. 150.]
78889 79630 [5 6]
2020-01-01T02:00:00.000000000 2020-01-31T23:00:00.000000000 [100. 150.]
78890 79631 [5 6]
2020-01-01T00:00:00.000000000 2020-01-31T21:00:00.000000000 [ 15.  30.  50.  75. 200. 250. 300. 400. 500.]
78888 79629 [ 1  2  3  4  7  8  9 10 11]
2020-01-01T01:00:00.000000000 2020-01-31T22:00:00.000000000 [ 15.  30.  50.  75. 200. 250. 300. 400. 500.]
78889 79630 [ 1  2  3  4  7  8  9 10 11]
2020-01-01T02:00:00.000000000 2020-01-31T23:00:00.000000000 [ 15.  30.  50.  75. 200. 250. 300. 400. 500.]
78890 7963

In [75]:
chunks = {'time':8}
dmy = xr.open_dataset('/media/harish/External_3/CERRA_ws_15_30_50_75_200_250_300_400_500/2020/CERRA_gridded_15_30_50_75_200_250_300_400_500_wind_2020_1.nc',
                chunks = chunks)
preprocess(dmy).load()

<xarray.Dataset>
Dimensions:            (time: 248, heightAboveGround: 9, y: 1069, x: 1069)
Coordinates:
  * time               (time) datetime64[ns] 2020-01-01 ... 2020-01-31T21:00:00
  * heightAboveGround  (heightAboveGround) float64 15.0 30.0 ... 400.0 500.0
Dimensions without coordinates: y, x
Data variables:
    ws                 (time, heightAboveGround, y, x) float32 8.033 ... 10.22
Attributes:
    GRIB_edition:            2
    GRIB_centre:             eswi
    GRIB_centreDescription:  Norrkoping
    GRIB_subCentre:          255
    Conventions:             CF-1.7
    institution:             Norrkoping
    history:                 2024-06-03T17:09 GRIB to CDM+CF via cfgrib-0.9.9...

In [77]:
chunks = {'valid_time':8}
dmy = xr.open_dataset('/media/harish/External_3/CERRA_ws_15_30_50_75_200_250_300_400_500_step1/2020/CERRA_gridded_15_30_50_75_200_250_300_400_500_wind_2020_1_1.nc'
                ,chunks = chunks)
dmy

<xarray.Dataset>
Dimensions:            (valid_time: 248, heightAboveGround: 9, y: 1069, x: 1069)
Coordinates:
  * valid_time         (valid_time) datetime64[ns] 2020-01-01T01:00:00 ... 20...
  * heightAboveGround  (heightAboveGround) float64 15.0 30.0 ... 400.0 500.0
    latitude           (y, x) float64 dask.array<chunksize=(1069, 1069), meta=np.ndarray>
    longitude          (y, x) float64 dask.array<chunksize=(1069, 1069), meta=np.ndarray>
    expver             (valid_time) object dask.array<chunksize=(8,), meta=np.ndarray>
Dimensions without coordinates: y, x
Data variables:
    ws                 (valid_time, heightAboveGround, y, x) float32 dask.array<chunksize=(8, 9, 1069, 1069), meta=np.ndarray>
Attributes:
    GRIB_centre:             eswi
    GRIB_centreDescription:  Norrkoping
    GRIB_subCentre:          255
    Conventions:             CF-1.7
    institution:             Norrkoping
    history:                 2024-12-18T15:24 GRIB to CDM+CF via cfgrib-0.9.1...

In [78]:
preprocess_2(dmy).load()

<xarray.Dataset>
Dimensions:            (time: 248, heightAboveGround: 9, y: 1069, x: 1069)
Coordinates:
  * time               (time) datetime64[ns] 2020-01-01T01:00:00 ... 2020-01-...
  * heightAboveGround  (heightAboveGround) float64 15.0 30.0 ... 400.0 500.0
Dimensions without coordinates: y, x
Data variables:
    ws                 (time, heightAboveGround, y, x) float32 7.832 ... 10.92
Attributes:
    GRIB_centre:             eswi
    GRIB_centreDescription:  Norrkoping
    GRIB_subCentre:          255
    Conventions:             CF-1.7
    institution:             Norrkoping
    history:                 2024-12-18T15:24 GRIB to CDM+CF via cfgrib-0.9.1...